<a href="https://colab.research.google.com/github/Naveensadanandan/Pytorch/blob/main/07_Email_classification_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import random

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler

from tqdm import tqdm
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/Datasets/translated_emails.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.head()

,content,language,queue,translated_text
0,Anfrage zu den Spezifikationen und Anpassungso...,de,Customer Service,Request for specifications and customization o...
1,Déconnexions fréquentes et plantages Le client...,fr,Product Support,Frequent Logouts and crashes The customer repo...
2,Problema de sonido Dell XPS Problema con el so...,es,Returns and Exchanges,"Dell XPS Sound Problem Sound Problem, handling..."
3,Assistance requise pour la configuration du ta...,fr,Product Support,Support required for the configuration of the ...
4,NaN,en,Returns and Exchanges,NaN


In [ ]:
df.dropna(inplace = True)

In [ ]:
label_encoder = LabelEncoder()
df["Department"] = label_encoder.fit_transform(df["queue"])
print(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

{'Billing and Payments': 0, 'Customer Service': 1, 'General Inquiry': 2, 'Human Resources': 3, 'IT Support': 4, 'Product Support': 5, 'Returns and Exchanges': 6, 'Sales and Pre-Sales': 7, 'Service Outages and Maintenance': 8, 'Technical Support': 9}


df.dropna(inplace=True)

In [ ]:
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize

def text_formatter(text : str):
    cleaned_text = " ".join(text.split())
    cleaned_text = cleaned_text.lower()
    # Remove URLs
    cleaned_text = re.sub(r'http\S+', '', cleaned_text)
    cleaned_text = re.sub(r'\d+','',cleaned_text)
    # Remove punctuation
    cleaned_text = cleaned_text.translate(str.maketrans('', '', string.punctuation))
    return cleaned_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df["processed_email"] = df["translated_text"].apply(text_formatter)

In [ ]:
from sklearn.model_selection import train_test_split
X = df[["processed_email","Department"]]
X_train, X_val = train_test_split(X, test_size=0.2, random_state=42)

# Convert Pandas Dataframe to hf dataset

In [ ]:
hg_train = Dataset.from_pandas(X_train)
hg_val = Dataset.from_pandas(X_val)

In [ ]:
hg_train, hg_val

(Dataset({
     features: ['processed_email', 'Department', '__index_level_0__'],
     num_rows: 414
 }),
 Dataset({
     features: ['processed_email', 'Department', '__index_level_0__'],
     num_rows: 104
 }))

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Load the pre-trained BERT model for sequence classification.
# 'bert-base-uncased' specifies the pre-trained BERT model to use.
# num_labels=2 indicates that the model is configured for a binary classification task.
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=10)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#defining a tokenize function
def tokenize_dataset(data):
  return tokenizer(
      data["processed_email"],
      padding = "max_length"
  )

train_dataset = hg_train.map(tokenize_dataset)
val_dataset = hg_val.map(tokenize_dataset)

Map:   0%|          | 0/414 [00:00<?, ? examples/s]

Map:   0%|          | 0/104 [00:00<?, ? examples/s]

In [ ]:
#removing unnecessary columns
train_dataset = train_dataset.remove_columns(["processed_email","__index_level_0__"])
val_dataset = val_dataset.remove_columns(["processed_email","__index_level_0__"])

train_dataset = train_dataset.rename_column("Department", "labels")
val_dataset = val_dataset.rename_column("Department", "labels")
train_dataset.set_format("torch")
val_dataset.set_format("torch")

In [ ]:
train_dataset, val_dataset

(Dataset({
     features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 414
 }),
 Dataset({
     features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 104
 }))

In [ ]:
#preparing dataloader
BATCH_SIZE = 4
train_dataloader = DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True
)

test_dataloader = DataLoader(
    dataset = val_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False
)

In [ ]:
next(iter(train_dataloader))

{'labels': tensor([9, 9, 5, 3]),
 'input_ids': tensor([[  101,  5057,  2490,  ...,     0,     0,     0],
         [  101, 24095, 19119,  ...,     0,     0,     0],
         [  101, 13661, 16360,  ...,     0,     0,     0],
         [  101,  3277,  2007,  ...,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
#initializing loss function and optmizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr = 3e-5)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
#Creating function for training and testing

def train_step(model : torch.nn.Module,
               loss_fn,
          dataloader : torch.utils.data.DataLoader,
          optimizer : torch.optim.Optimizer):
  model.to(device)
  model.train()

  train_loss, train_acc = 0,0
  for batch in dataloader:
    batch = {k:v.to(device) for k,v in batch.items()}
    labels = batch["labels"]

    outputs = model(**batch)
    logits = outputs.logits

    loss = loss_fn(logits, labels)
    train_loss += loss

    optimizer.zero_grad()

    loss.backward()
    optimizer.step()

    y_pred_class = torch.argmax(torch.softmax(logits, dim=1), dim=1)
    train_acc += (y_pred_class == labels).sum().item()/len(y_pred_class)
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

In [ ]:
def test_step(model:torch.nn.Module,
             dataloader:torch.utils.data.DataLoader):
    model.eval()

    test_loss, test_acc = 0, 0
    with torch.inference_mode():
        for batch in dataloader:
            batch = {k:v.to(device) for k,v in batch.items()}
            labels = batch["labels"]

            # Forward pass
            logits = model(**batch)
            y_pred = logits.logits
            loss = logits.loss
            test_loss += loss.item()

            test_pred_label = torch.argmax(torch.softmax(y_pred, dim =1), dim=1)
            test_acc += (test_pred_label == labels).sum().item()/len(y_pred)

    test_loss = test_loss/len(dataloader)
    test_acc = test_acc/len(dataloader)
    return test_loss, test_acc

In [ ]:
from tqdm.auto import tqdm
from torch import nn

# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module,
          loss_fn,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          epochs: int = 5):

    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }

    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           loss_fn=loss_fn,
                                           dataloader=train_dataloader,
                                           optimizer=optimizer)
        test_loss, test_acc = test_step(model=model,
            dataloader=test_dataloader)

        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # 5. Update results dictionary
        # Ensure all data is moved to CPU and converted to float for storage
        results["train_loss"].append(train_loss.item() if isinstance(train_loss, torch.Tensor) else train_loss)
        results["train_acc"].append(train_acc.item() if isinstance(train_acc, torch.Tensor) else train_acc)
        results["test_loss"].append(test_loss.item() if isinstance(test_loss, torch.Tensor) else test_loss)
        results["test_acc"].append(test_acc.item() if isinstance(test_acc, torch.Tensor) else test_acc)

    # 6. Return the filled results at the end of the epochs
    return results

In [ ]:
#training and validation
EPOCHS = 5
torch.manual_seed(42)
torch.cuda.manual_seed(42)
model_results = train(model=model,
                      loss_fn = loss_function,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        optimizer=optimizer,
                        epochs=EPOCHS)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.8891 | train_acc: 0.3702 | test_loss: 1.7186 | test_acc: 0.3654
Epoch: 2 | train_loss: 1.3351 | train_acc: 0.5505 | test_loss: 1.1414 | test_acc: 0.5769
Epoch: 3 | train_loss: 0.7067 | train_acc: 0.7861 | test_loss: 0.7010 | test_acc: 0.7885
Epoch: 4 | train_loss: 0.2642 | train_acc: 0.9639 | test_loss: 0.4687 | test_acc: 0.8942
Epoch: 5 | train_loss: 0.1067 | train_acc: 0.9952 | test_loss: 0.3213 | test_acc: 0.9327


In [ ]:
def prediction(text):
  input_ids = tokenizer(text)
  logits = model(**input_ids)
  return logits

In [ ]:
def prediction_pipe(text):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt").to(device)
    with torch.inference_mode():
      outputs = model(**inputs)

      logits = outputs.logits

      # Apply softmax to get probabilities
      probs = torch.nn.functional.softmax(logits, dim=-1)

      # Get the predicted class
      predicted_class_idx = torch.argmax(probs, dim=-1).item()

      # Map the predicted class index to the corresponding label
      predicted_class = label_encoder.inverse_transform([predicted_class_idx]).item()

      return predicted_class

In [ ]:
m= '''
Dear Sir,
I am writing this mail to infrom you that i didn't get  a satified response from your team regarding the issue that i raised four days ago.
My ticket number is TYU458368.
Please take necessary actions at earliest.
Regards
'''

prediction_pipe(m)

'Technical Support'

In [ ]:
mail = '''
Dear IT Support Team,

I hope this message finds you well. I am writing to request assistance with the installation of the new software tool, Project Management Pro, on my workstation. I have been having trouble with the installation process, and it seems that I may not have the necessary permissions to complete it.

Could you please help me with the installation at your earliest convenience? If needed, I can be available for a quick remote session to resolve this issue.

Thank you for your support!
'''
prediction_pipe(mail)

'Technical Support'

In [ ]:
mail = '''
Dell XPS Sound Problem Sound Problem, handling as a return. Within 30 days.
'''
prediction_pipe(mail)

'Returns and Exchanges'

In [ ]:
emails = [
    "I have a question regarding my billing statement.",
    "Could you please help me with my order status?",
    "I would like to return my recent purchase.",
    "Is there a technical issue with the software?",
    "How can I exchange an item I bought?"
]

# Create a DataFrame with the sample emails
df_test = pd.DataFrame({'email_content': emails})
df_test["predictions"] = df_test["email_content"].apply(prediction_pipe)
df_test

,email_content,predictions
0,I have a question regarding my billing statement.,Billing and Payments
1,Could you please help me with my order status?,Customer Service
2,I would like to return my recent purchase.,Returns and Exchanges
3,Is there a technical issue with the software?,Technical Support
4,How can I exchange an item I bought?,Returns and Exchanges


In [ ]:
# Save the model and tokenizer to Google Drive
model.save_pretrained('/content/drive/MyDrive/saved_model')
tokenizer.save_pretrained('/content/drive/MyDrive/saved_model')

('/content/drive/MyDrive/saved_model/tokenizer_config.json',
 '/content/drive/MyDrive/saved_model/special_tokens_map.json',
 '/content/drive/MyDrive/saved_model/vocab.txt',
 '/content/drive/MyDrive/saved_model/added_tokens.json',
 '/content/drive/MyDrive/saved_model/tokenizer.json')

In [ ]:
# Save the label encoder
import joblib
joblib.dump(label_encoder, '/content/drive/MyDrive/saved_model/label_encoder.pkl')

['/content/drive/MyDrive/saved_model/label_encoder.pkl']